In [9]:
# Dependencies
# scipy=0.17.0
# nltk=3.2.1
from nltk.corpus import wordnet
import nltk
import string
import scipy
from scipy import spatial
import Levenshtein
from Levenshtein.StringMatcher import StringMatcher as SequenceMatcher

### Upload Glove vectors

In [2]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

model=loadGloveModel("glove.6B.300d.txt")

Loading Glove Model
Done. 400000  words loaded!


### Create definitions

In [5]:
def sent_to_word2vec(statement, other_statement):
    
    a = statement.lower()
    b = other_statement.lower()
    #a=re.sub("\d+", "", statement.lower())
    #b=re.sub("\d+", "", other_statement.lower())
    
    # Get default English stopwords and extend with punctuation
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(string.punctuation)
    stopwords.append('')
    #remove_months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 
                # 'september', 'october', 'november', 'december']
    #remove_days=['monday', 'tuesday', 'wednesday', 'thursday','friday', 'saturday','sunday']
    #stopwords.extend(remove_months[:])
    #stopwords.extend(remove_days[:])
    
    
    # Transform sentences to vectors with exception if word not on list (include stop words if list=0)  
    def sentence_vector(statement):
        list=[]
        full_list = []
        for i in nltk.tokenize.word_tokenize(statement):
            try:
                full_list.append(model[i])
                if i not in stopwords:
                    list.append(model[i])
            except KeyError:
                continue
        if len(list) == 0:
            return full_list
        else:
            return list
    
    
    
    list_of_lists1=sentence_vector(a)
    list_of_lists2=sentence_vector(b)
    #print(len(lists_of_lists1),len(lists_of_lists2))
            
    # Sum of the word vectors to create a sentence_vector
    a_v=[sum(x) for x in zip(*list_of_lists1)]
    b_v=[sum(x) for x in zip(*list_of_lists2)]

    
    # Calculate cosine similarity
    if (len(a_v)==0) or (len(b_v)==0):
        result = 0.0
    else:
        result = 1 - spatial.distance.cosine(a_v, b_v)
    
    return float("{0:.3f}".format(result))
    #return [len(list_of_lists1),len(list_of_lists2)]

    
    
def sentence_vector(statement):
        a=statement.lower()
        #a=re.sub("\d+", "", statement.lower())
        
        stopwords = nltk.corpus.stopwords.words('english')
        stopwords.extend(string.punctuation)
        stopwords.append('')
        #remove_months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 
         #        'september', 'october', 'november', 'december']
        #remove_days=['monday', 'tuesday', 'wednesday', 'thursday','friday', 'saturday','sunday']
        #stopwords.extend(remove_months[:])
        #stopwords.extend(remove_days[:])
        
        list=[]
        full_list = []
        for i in nltk.tokenize.word_tokenize(a):
            try:
                full_list.append(i)
                if i not in stopwords:
                    list.append(i)
            except KeyError:
                continue
        if len(list) == 0:
            return full_list
        else:
            return list


###  Incomplete information 

### Setup

Example:
Customers asks:  What are the baggage fees in KLM?
Bot response: 

[scenario 1: Bot does not have that statement in its database (it needs to know if you are traveling to US, Europe or within Canada). Bot figures out there is some info missing in the question. Then Bot asks for more info and joins both statements to finally look for a similar statement in its database]

[scenario 2: We added a similar question to the database. Bot finds it. The answer to that question is to ask for more info. Once the additional info is provided , the bot provides the right answer] 


Two alternative solutions:

(1) Scenario 1: If bot can not find another similar statement in the database with certainty, then find out if there is missing info and act accordingly. See code below for prototype using word2vec and Levenshtein.

(2) Scenario 2: This scenario basically requires to add statements to the database and create convearsations that match the kind of question the customer asks ("What are the baggage fees in KLM?"). It requires to add more statements/ conversations to the database

See datasets "baggage_example1" and "baggage_example2" defined below

#### Scenario 1 : There is no similar question in the database

In [28]:

baggage_example1= {
    "baggage_example": [
        [          
            "What are Air Canada's  baggage fees traveling in Canada ?",
            "USD 15 for 1st bag, USD30 for second bag. USD 100 for additional bags"
        ],
        [            
            "What are Air Canada's  baggage fees traveling to US ?",
            "USD 20 for 1st bag, USD40 for second bag. USD 100 for additional bags"
        ],
        [    
            "What are Air Canada's  baggage fees traveling to Europe ?",
            "USD 25 for 1st bag, USD45 for second bag. USD 100 for additional bags"
        ],
        [          
            "What are American Airlines'  baggage fees traveling in Canada ?",
            "USD 17 for 1st bag, USD32 for second bag. USD 100 for additional bags"
        ],
        [            
            "What are American Airlines'  baggage fees traveling to US ?",
            "USD 18 for 1st bag, USD37 for second bag. USD 100 for additional bags"
        ],
        [    
            "What are American Airlines'  baggage fees traveling to Europe ?",
            "USD 23 for 1st bag, USD42 for second bag. USD 100 for additional bags"
        ],
        [          
            "What are KLM's  baggage fees traveling in Canada ?",
            "USD 22.5 for 1st bag, USD32.5 for second bag. USD 100 for additional bags"
        ],
        [            
            "What are KLM's  baggage fees traveling to US ?",
            "USD 18.5 for 1st bag, USD37.5 for second bag. USD 100 for additional bags"
        ],
        [    
            "What are KLM's  baggage fees traveling to Europe ?",
            "USD 23.5 for 1st bag, USD42.5 for second bag. USD 100 for additional bags"
        ],
        [    
            "Are you traveling in Canada?",
            "Programmed action 1 (join sentences and analyze)"
        ],
        [    
            "Are you traveling to US?",
            "Programmed action 2 (join sentences and analyze)"
        ],
        [    
            "Are you traveling to Europe?",
            "Programmed action 3 (join sentences and analyze)"
        ],
        [    
            "Are traveling on Air Canada?",
            "Programmed action 4 (join sentences and analyze)"
        ],
        [    
            "Where are you traveling to (Europe, US, within Canada)?",
            "Programmed action 5 (join sentences and analyze)"
        ]
    ]
}
          

In [29]:
# Find the most similar sentence in the baggage_example1 dictionary
# "What are KLM's  baggage fees traveling to Europe ? " is the most similar statement 
# but only obtains 0.8 in the cosine similarity (meaning is not really that close, more than 0.9 is close)

selected_sentence="What are the baggage fees in KLM?"

a= " ".join(sentence_vector(selected_sentence))

for i in baggage_example1['baggage_example']:
    b= " ".join(sentence_vector(i[0]))
    cosine=sent_to_word2vec(selected_sentence,i[0])
    leven=float("{0:.3f}".format(SequenceMatcher(None,a,b).ratio()))
    print(cosine, " ",leven," ", i[0])


0.577   0.475   What are Air Canada's  baggage fees traveling in Canada ?
0.595   0.509   What are Air Canada's  baggage fees traveling to US ?
0.611   0.475   What are Air Canada's  baggage fees traveling to Europe ?
0.7   0.444   What are American Airlines'  baggage fees traveling in Canada ?
0.684   0.475   What are American Airlines'  baggage fees traveling to US ?
0.708   0.444   What are American Airlines'  baggage fees traveling to Europe ?
0.793   0.538   What are KLM's  baggage fees traveling in Canada ?
0.792   0.583   What are KLM's  baggage fees traveling to US ?
0.8   0.538   What are KLM's  baggage fees traveling to Europe ?
0.194   0.188   Are you traveling in Canada?
0.193   0.286   Are you traveling to US?
0.224   0.312   Are you traveling to Europe?
0.304   0.222   Are traveling on Air Canada?
0.219   0.286   Where are you traveling to (Europe, US, within Canada)?


In [33]:
# Substract words from "What are KLM's  baggage fees traveling to Europe ?" if they are 
# in the customer's statement ("What are the baggage fees in KLM?")

new_list=sentence_vector(baggage_example1['baggage_example'][8][0])
for i in sentence_vector(selected_sentence): 
    new_list= [x for x in new_list if x != i ]
print(" ".join(new_list))

's traveling europe


In [31]:
# Find the most similar sentence to the words "'s traveling europe"
# "Are you traveling to Europe?" is the most similar statement. Bot asks that and waits for answer


selected_sentence2=" ".join(new_list)
a= " ".join(sentence_vector(selected_sentence))

for i in baggage_example1['baggage_example']:
    b= " ".join(sentence_vector(i[0]))
    cosine=sent_to_word2vec(selected_sentence2,i[0])
    leven=float("{0:.3f}".format(SequenceMatcher(None,a,b).ratio()))
    print(cosine, " ",leven," ", i[0])

0.705   0.475   What are Air Canada's  baggage fees traveling in Canada ?
0.731   0.509   What are Air Canada's  baggage fees traveling to US ?
0.807   0.475   What are Air Canada's  baggage fees traveling to Europe ?
0.589   0.444   What are American Airlines'  baggage fees traveling in Canada ?
0.578   0.475   What are American Airlines'  baggage fees traveling to US ?
0.665   0.444   What are American Airlines'  baggage fees traveling to Europe ?
0.669   0.538   What are KLM's  baggage fees traveling in Canada ?
0.672   0.583   What are KLM's  baggage fees traveling to US ?
0.754   0.538   What are KLM's  baggage fees traveling to Europe ?
0.691   0.188   Are you traveling in Canada?
0.694   0.286   Are you traveling to US?
0.888   0.312   Are you traveling to Europe?
0.678   0.222   Are traveling on Air Canada?
0.818   0.286   Where are you traveling to (Europe, US, within Canada)?


In [42]:
# The joins answer to original question asked by customer
# customer answers: "NO. I am traveling to the US"
# the most similar question to the combined sentence (selected_sentence3 below) is 
# "What are KLM's  baggage fees traveling to US ?" which is in our database and we have an answer for.

selected_sentence3="What are the baggage fees in KLM? No. I am traveling to the US"

a= " ".join(sentence_vector(selected_sentence3))

for i in baggage_example1['baggage_example']:
    b= " ".join(sentence_vector(i[0]))
    cosine=sent_to_word2vec(selected_sentence3,i[0])
    leven=float("{0:.3f}".format(SequenceMatcher(None,a,b).ratio()))
    print(cosine, " ",leven," ", i[0])

0.747   0.639   What are Air Canada's  baggage fees traveling in Canada ?
0.835   0.735   What are Air Canada's  baggage fees traveling to US ?
0.797   0.667   What are Air Canada's  baggage fees traveling to Europe ?
0.854   0.605   What are American Airlines'  baggage fees traveling in Canada ?
0.901   0.694   What are American Airlines'  baggage fees traveling to US ?
0.871   0.632   What are American Airlines'  baggage fees traveling to Europe ?
0.879   0.708   What are KLM's  baggage fees traveling in Canada ?
0.957   0.82   What are KLM's  baggage fees traveling to US ?
0.896   0.738   What are KLM's  baggage fees traveling to Europe ?
0.504   0.444   Are you traveling in Canada?
0.67   0.585   Are you traveling to US?
0.56   0.489   Are you traveling to Europe?
0.586   0.408   Are traveling on Air Canada?
0.588   0.387   Where are you traveling to (Europe, US, within Canada)?


#### Scenario 2 : There is a similar question in the database

In [16]:

baggage_example2= {
    "baggage_example": [
        [          
            "What are Air Canada's  baggage fees traveling in Canada ?",
            "USD 15 for 1st bag, USD30 for second bag. USD 100 for additional bags"
        ],
        [            
            "What are Air Canada's  baggage fees traveling to US ?",
            "USD 20 for 1st bag, USD40 for second bag. USD 100 for additional bags"
        ],
        [    
            "What are Air Canada's  baggage fees traveling to Europe ?",
            "USD 25 for 1st bag, USD45 for second bag. USD 100 for additional bags"
        ],
        [          
            "What are American Airlines'  baggage fees traveling in Canada ?",
            "USD 17 for 1st bag, USD32 for second bag. USD 100 for additional bags"
        ],
        [            
            "What are American Airlines'  baggage fees traveling to US ?",
            "USD 18 for 1st bag, USD37 for second bag. USD 100 for additional bags"
        ],
        [    
            "What are American Airlines'  baggage fees traveling to Europe ?",
            "USD 23 for 1st bag, USD42 for second bag. USD 100 for additional bags"
        ],
        [          
            "What are KLM's  baggage fees traveling in Canada ?",
            "USD 22.5 for 1st bag, USD32.5 for second bag. USD 100 for additional bags"
        ],
        [            
            "What are KLM's  baggage fees traveling to US ?",
            "USD 18.5 for 1st bag, USD37.5 for second bag. USD 100 for additional bags"
        ],
        [    
            "What are KLM's  baggage fees traveling to Europe ?",
            "USD 23.5 for 1st bag, USD42.5 for second bag. USD 100 for additional bags"
        ],
        [    
            "Are you traveling in Canada?",
            "Programmed action 1 (join sentences and analyze)"
        ],
        [    
            "Are you traveling to US?",
            "Programmed action 2 (join sentences and analyze)"
        ],
        [    
            "Are you traveling to Europe?",
            "Programmed action 3 (join sentences and analyze)"
        ],
        [    
            "Are traveling on Air Canada?",
            "Programmed action 4 (join sentences and analyze)"
        ],
        [    
            "Are traveling on American Airlines?",
            "Programmed action 5 (join sentences and analyze)"
        ],
        [    
            "Are traveling on KLM?",
            "Programmed action 6 (join sentences and analyze)"
        ],
        [    
            "What are the baggage fees when you travel in KLM?",
            "Answer: Where are you traveling to [Canada, US, Europe]?"
        ],
        [    
            "I am going to Europe",
            "Answer: Link to previous statement, join and analize"
        ]
    ]
}
          

In [43]:
# If the statement is in the database, then both word2vec and Levenshtein find the right statement

selected_sentence="What are the baggage fees in KLM?"

a= " ".join(sentence_vector(selected_sentence))

for i in baggage_example2['baggage_example']:
    b= " ".join(sentence_vector(i[0]))
    cosine=sent_to_word2vec(selected_sentence,i[0])
    leven=float("{0:.3f}".format(SequenceMatcher(None,a,b).ratio()))
    print(cosine, " ",leven," ", i[0])


0.577   0.475   What are Air Canada's  baggage fees traveling in Canada ?
0.595   0.509   What are Air Canada's  baggage fees traveling to US ?
0.611   0.475   What are Air Canada's  baggage fees traveling to Europe ?
0.7   0.444   What are American Airlines'  baggage fees traveling in Canada ?
0.684   0.475   What are American Airlines'  baggage fees traveling to US ?
0.708   0.444   What are American Airlines'  baggage fees traveling to Europe ?
0.793   0.538   What are KLM's  baggage fees traveling in Canada ?
0.792   0.583   What are KLM's  baggage fees traveling to US ?
0.8   0.538   What are KLM's  baggage fees traveling to Europe ?
0.194   0.188   Are you traveling in Canada?
0.193   0.286   Are you traveling to US?
0.224   0.312   Are you traveling to Europe?
0.304   0.222   Are traveling on Air Canada?
0.424   0.326   Are traveling on American Airlines?
0.586   0.414   Are traveling on KLM?
0.941   0.821   What are the baggage fees when you travel in KLM?
0.21   0.357   I am g